In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from IPython.display import display, clear_output
import math


## Setup MediaPipe


In [ ]:
mp_hands = mp.solutions.hands
# Diminuir confidence para melhorar a detecção
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Função para garantir que a câmera está funcionando
def check_camera():
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        print("ERRO: Não foi possível acessar a câmera!")
        return False
    camera.release()
    return True

# Verificar câmera
if not check_camera():
    print("Utilize a opção abaixo para modo de demonstração sem câmera")

def create_graph():
    G = nx.random_geometric_graph(10, 0.5, dim=3)
    pos = nx.get_node_attributes(G, 'pos')
    for k, v in pos.items():
        G.nodes[k]['pos'] = np.array(v)
    return G

def plot_3d_graph(G, camera=None):
    import plotly.io as pio
    # Configurar plotly para melhor performance em notebook
    pio.renderers.default = "notebook_connected"

    # Extrair posições dos nós
    pos = nx.get_node_attributes(G, 'pos')

    # Criar as coordenadas dos nós
    Xn = [pos[k][0] for k in G.nodes()]
    Yn = [pos[k][1] for k in G.nodes()]
    Zn = [pos[k][2] for k in G.nodes()]

    # Criar as coordenadas das arestas
    Xe, Ye, Ze = [], [], []
    for e in G.edges():
        Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
        Ye.extend([pos[e[0]][1], pos[e[1]][1], None])
        Ze.extend([pos[e[0]][2], pos[e[1]][2], None])

    # Criar o grafo 3D
    trace_nodes = go.Scatter3d(x=Xn, y=Yn, z=Zn,
                               mode='markers',
                               marker=dict(size=8, color='blue'),
                               hoverinfo='text',
                               text=[f'Node {i}' for i in G.nodes()])

    trace_edges = go.Scatter3d(x=Xe, y=Ye, z=Ze,
                               mode='lines',
                               line=dict(width=1, color='black'),
                               hoverinfo='none')

    layout = go.Layout(
        title='Grafo 3D Interativo',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            camera=camera if camera else dict(eye=dict(x=1.25, y=1.25, z=1.25))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig = go.Figure(data=[trace_edges, trace_nodes], layout=layout)
    return fig


## Funções para detecção de gestos


In [ ]:
def is_hand_open(hand_landmarks):
    # Compara a posição da ponta dos dedos com as juntas inferiores para verificar se estão estendidos
    # Polegar (posição diferente por causa da orientação)
    thumb_tip = hand_landmarks.landmark[4].y < hand_landmarks.landmark[3].y

    # Outros dedos - verificar se estão estendidos (ponta acima da junta inferior)
    index_finger = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    middle_finger = hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y
    ring_finger = hand_landmarks.landmark[16].y < hand_landmarks.landmark[14].y
    pinky = hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y

    # Mão é considerada aberta se a maioria dos dedos estiver estendida
    fingers_extended = sum([thumb_tip, index_finger, middle_finger, ring_finger, pinky])
    return fingers_extended >= 4

def is_hand_closed(hand_landmarks):
    # Mão fechada: maioria dos dedos dobrados
    thumb_folded = hand_landmarks.landmark[4].y > hand_landmarks.landmark[3].y
    index_folded = hand_landmarks.landmark[8].y > hand_landmarks.landmark[6].y
    middle_folded = hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y
    ring_folded = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    pinky_folded = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y

    fingers_folded = sum([thumb_folded, index_folded, middle_folded, ring_folded, pinky_folded])
    return fingers_folded >= 4

def measure_pinch_distance(hand_landmarks):
    # Calcular a distância entre as pontas do polegar e do indicador
    thumb_tip = np.array([hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, hand_landmarks.landmark[4].z])
    index_tip = np.array([hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y, hand_landmarks.landmark[8].z])
    return np.linalg.norm(thumb_tip - index_tip)


## Inicia câmera e manipulação


In [ ]:
cap = cv2.VideoCapture(0)
prev_x, prev_y = None, None
camera_eye = dict(x=1.25, y=1.25, z=1.25)
G = create_graph()

# Aumentar sensibilidade para melhorar resposta dos gestos
rotation_speed = 0.05  # Aumentado de 0.02
movement_speed = 0.1   # Aumentado de 0.05
zoom_sensitivity = 10.0  # Aumentado de 5.0

# Variáveis para rastreamento de gestos
prev_hand_pos = None
prev_pinch_distance = None

# Configuração para atualização do grafo
update_graph = False

# Configurações iniciais para o grafo - usando 'notebook_connected' como renderer
fig = plot_3d_graph(G, dict(eye=camera_eye))
fig.show(renderer="notebook_connected")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        h, w, _ = frame.shape

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]

            # Extrair posição da mão (usar ponto central da mão)
            wrist = hand_landmarks.landmark[0]
            hand_pos = np.array([wrist.x, wrist.y])

            # Detectar o tipo de gesto
            hand_open = is_hand_open(hand_landmarks)
            hand_closed = is_hand_closed(hand_landmarks)
            pinch_distance = measure_pinch_distance(hand_landmarks)

            if prev_hand_pos is not None:
                # Calcular movimento da mão
                dx = hand_pos[0] - prev_hand_pos[0]
                dy = hand_pos[1] - prev_hand_pos[1]

                # 1. Controle com mão aberta: rotacionar o grafo
                if hand_open:
                    camera_eye['x'] += dy * rotation_speed
                    camera_eye['y'] += dx * rotation_speed
                    update_graph = True

                # 2. Controle com mão fechada: mover no eixo X
                elif hand_closed:
                    # Mover a visualização
                    camera_eye['x'] += dx * movement_speed
                    camera_eye['z'] += dy * movement_speed
                    update_graph = True

                # 3. Controle pinça (polegar + indicador): zoom
                if prev_pinch_distance is not None:
                    # Calcular diferença de distância para zoom
                    zoom_delta = prev_pinch_distance - pinch_distance

                    # Aplicar zoom (alterar a distância da câmera)
                    zoom_factor = 1 + (zoom_delta * zoom_sensitivity)
                    camera_eye['x'] *= zoom_factor
                    camera_eye['y'] *= zoom_factor
                    camera_eye['z'] *= zoom_factor

                    if abs(zoom_delta) > 0.01:
                        update_graph = True

            # Atualizar posições anteriores
            prev_hand_pos = hand_pos
            prev_pinch_distance = pinch_distance

            # Desenhar landmarks da mão
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Adicionar texto informativo
            status_text = "Mão aberta: rotação | Mão fechada: movimento | Pinça: zoom"
            cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Indicar o gesto detectado
            gesture = "Gesto: "
            if hand_open:
                gesture += "Mão aberta (rotação)"
            elif hand_closed:
                gesture += "Mão fechada (movimento)"
            else:
                gesture += "Pinça (zoom)"
            cv2.putText(frame, gesture, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            prev_hand_pos = None
            prev_pinch_distance = None

        # Mostrar valores de debug para verificar se os gestos estão sendo detectados
        if results.multi_hand_landmarks:
            debug_text = f"Hand: {'Aberta' if hand_open else 'Fechada' if hand_closed else 'Neutra'} | dx: {dx:.4f}, dy: {dy:.4f}"
            cv2.putText(frame, debug_text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Atualizar o grafo se necessário
        if update_graph:
            # Usar método alternativo para atualização do grafo sem limpar a saída
            # Isso evita problemas de renderização no notebook
            from IPython.display import display
            # Não usar clear_output pois pode causar flickering
            fig = plot_3d_graph(G, dict(eye=camera_eye))
            # Usar renderer conectado para melhor interatividade
            fig.show(renderer="notebook_connected")
            update_graph = False

        # Mostrar frame da câmera
        cv2.imshow("Camera - Controle do grafo - Pressione 'q' para sair", frame)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Erro: {e}")
finally:
    cap.release()
    cv2.destroyAllWindows()